In [1]:
import pandas as pd
df=pd.read_csv('Churn_Modelling.csv')

In [2]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Gender']=le.fit_transform(df[['Gender']])

d:\projects\DeepLearningProject\dpenv\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
geo_ohe=ohe.fit_transform(df[['Geography']])

In [7]:
geo_ohe.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
df_geo=pd.DataFrame(geo_ohe.toarray(),columns=ohe.get_feature_names_out(['Geography']))

In [9]:
df=pd.concat([df.drop(['Geography'],axis=1),df_geo],axis=1)

In [10]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
x=df.drop('Exited',axis=1)
y=df['Exited']

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler
scalar=StandardScaler()
x_train=scalar.fit_transform(x_train)
x_test=scalar.transform(x_test)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard

In [15]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(35,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [16]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [17]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [18]:
#set up tensorflow board
import datetime
log_dir='logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [19]:
# set up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [20]:
# Train the model
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100


235/235 [==============================] - 7s 14ms/step - loss: 0.4592 - accuracy: 0.7981 - val_loss: 0.4025 - val_accuracy: 0.8272
Epoch 2/100
235/235 [==============================] - 1s 6ms/step - loss: 0.3917 - accuracy: 0.8356 - val_loss: 0.3685 - val_accuracy: 0.8460
Epoch 3/100
235/235 [==============================] - 2s 7ms/step - loss: 0.3618 - accuracy: 0.8497 - val_loss: 0.3534 - val_accuracy: 0.8576
Epoch 4/100
235/235 [==============================] - 2s 10ms/step - loss: 0.3482 - accuracy: 0.8571 - val_loss: 0.3504 - val_accuracy: 0.8624
Epoch 5/100
235/235 [==============================] - 1s 6ms/step - loss: 0.3430 - accuracy: 0.8600 - val_loss: 0.3490 - val_accuracy: 0.8608
Epoch 6/100
235/235 [==============================] - 2s 7ms/step - loss: 0.3387 - accuracy: 0.8593 - val_loss: 0.3515 - val_accuracy: 0.8584
Epoch 7/100
235/235 [==============================] - 2s 8ms/step - loss: 0.3359 - accuracy: 0.8591 - val_loss: 0.3445 - val_accuracy: 0.

In [21]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 13896), started 16:57:56 ago. (Use '!kill 13896' to kill it.)

In [ ]:
# import pickle
# with open('label_encoder_gender.pkl','wb') as f:
#   pickle.dump(le,f)

# with open('onehot_encoder_geography.pkl','wb') as f:
#   pickle.dump(ohe,f)

In [24]:
with open('scalar.pkl','wb') as f:
  pickle.dump(scalar,f)

In [25]:
model.save('churn_model.h5')

d:\projects\DeepLearningProject\dpenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
input_data={
  'CreditScore':600,
  'Geography':'France',
  'Gender':'Male',
  'Age':40,
  'Tenure':3,
  'Balance':60000,
  'NumOfProducts':2,
  'HasCrCard':1,
  'IsActiveMember':1,
  'EstimatedSalary':50000
}

In [27]:
from tensorflow.keras.models import load_model
model=load_model('churn_model.h5')

In [29]:
import joblib

# After fitting your encoders and scaler:
joblib.dump(ohe, 'onehot_encoder_geography.pkl')
joblib.dump(le, 'label_encoder_gender.pkl')
joblib.dump(scalar, 'scalar.pkl')

['scalar.pkl']

In [ ]:
# with open('label_encoder_gender.pkl','rb') as f:
#   le=pickle.load(f)
# with open('onehot_encoder_geography.pkl','rb') as f:
#   ohe=pickle.load(f)
# with open('scalar.pkl','rb') as f:
#   scalar=pickle.load(f)

In [ ]:
input_df=pd.DataFrame([input_data])

In [ ]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [ ]:
input_df['Gender']=le.transform(input_df[['Gender']])

d:\projects\DeepLearningProject\dpenv\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [ ]:
ohe_geography=ohe.transform(input_df[['Geography']])
df2=pd.DataFrame(ohe_geography.toarray(),columns=ohe.get_feature_names_out(['Geography']))

In [ ]:
input_df=pd.concat([input_df.drop(['Geography'],axis=1),df2],axis=1)

In [ ]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
scaled_input=scalar.transform(input_df)

In [ ]:
pred=model.predict(scaled_input)

1/1 [==============================] - 0s 184ms/step


In [ ]:
pred

array([[0.04542109]], dtype=float32)

In [ ]:
pred[0][0]

0.045421094

In [ ]:
if pred[0][0]>0.5:
  print("The customer is likely to churn.")
else:
  print("The customer is not likely to churn.")

The customer is not likely to churn.
